In [1]:
!pip install kaggle

Before continue please upload your kaggle.json credential

In [2]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Download datasets with API Command from kaggle

In [3]:
!kaggle datasets download -d columbine/imdb-dataset-sentiment-analysis-in-csv-format

 90% 23.0M/25.7M [00:00<00:00, 57.9MB/s]
100% 25.7M/25.7M [00:00<00:00, 52.6MB/s]


In [4]:
!unzip imdb-dataset-sentiment-analysis-in-csv-format.zip -d ./data

Archive:  imdb-dataset-sentiment-analysis-in-csv-format.zip
  inflating: ./data/Test.csv         
  inflating: ./data/Train.csv        
  inflating: ./data/Valid.csv        


In [5]:
import pandas as pd

# Load the training dataset
df = pd.read_csv('/content/data/Train.csv')

df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [6]:
df.describe()

,label
count,40000.000000
mean,0.499525
std,0.500006
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [20]:
import csv

training_sentences = []
training_labels = []

validation_sentences = []
validation_labels = []

with open('./data/Train.csv') as csvfile:
     reader = csv.reader(csvfile, delimiter=',')
     next(reader)
     for row in reader:
         training_sentences.append(row[0])
         training_labels.append(row[1])


with open('./data/Valid.csv') as csvfile:
     reader = csv.reader(csvfile, delimiter=',')
     next(reader)
     for row in reader:
         validation_sentences.append(row[0])
         validation_labels.append(row[1])

In [21]:
print(training_sentences[0])
print(training_labels[0])

print(validation_sentences[0])
print(validation_labels[0])

I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.
0
It's been about 14 years since Sharon Stone awarded viewers a leg-crossing that twisted many people's minds. And now, God k

In [28]:
training_labels = np.array(training_labels, dtype=np.int32)
validation_labels = np.array(validation_labels, dtype=np.int32)

In [29]:
vocab_size = 10000
embedding_dim = 32
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_portion = .8

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np

In [31]:
# Fit the tokenizer with training data
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=trunc_type, truncating=trunc_type)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=trunc_type, truncating=trunc_type)

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Make sure we are using "sparse_categorical_crossentropy" as a loss fuction
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(padded, training_labels, epochs=10, validation_data=(validation_padded, validation_labels), callbacks=[])

Epoch 1/10
1250/1250 [==============================] - 13s 9ms/step - loss: 0.5597 - accuracy: 0.7449 - val_loss: 0.4404 - val_accuracy: 0.8154
Epoch 2/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.3785 - accuracy: 0.8429 - val_loss: 0.3712 - val_accuracy: 0.8360
Epoch 3/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.3226 - accuracy: 0.8673 - val_loss: 0.3494 - val_accuracy: 0.8410
Epoch 4/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2920 - accuracy: 0.8804 - val_loss: 0.3420 - val_accuracy: 0.8458
Epoch 5/10
1250/1250 [==============================] - 10s 8ms/step - loss: 0.2719 - accuracy: 0.8909 - val_loss: 0.3394 - val_accuracy: 0.8494
Epoch 6/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2568 - accuracy: 0.8976 - val_loss: 0.3423 - val_accuracy: 0.8504
Epoch 7/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2456 - accuracy: 0.9029 - val_loss: 0.3462 - val_ac